In [8]:
%pip install -qU langchain langchain-openai langchain-anthropic langsmith python-dotenv pydantic

Note: you may need to restart the kernel to use updated packages.


In [5]:
from dotenv import load_dotenv
load_dotenv()

from langsmith import Client

client = Client()


In [6]:
example_system_prompt = """You are a helpful assistant

Rules:
- You are only allowed to talk about coding
- <HERE>
- Try to be concise"""

example_instruction = "The agent should only respond in English. No other languages"

examples = ["Hola, como estas?", "Hi there!"]

In [7]:
from typing import List
import random
from langsmith.schemas import Run, Example
from langsmith import evaluate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from pydantic.v1 import BaseModel, Field
from langchain.chat_models import init_chat_model

# Schemas


class Rule(BaseModel):
    """A single rule for the prompt"""
    reasoning: str = Field(
        ..., description="The thought process and direction for why we think this is a good solution to the instruction.")
    prompt: str = Field(..., description="A single prompt rule that we can try to solve for the instruction. \
This prompt rule will be interpolated into the system prompt over the <HERE> token.")


class Rules(BaseModel):
    """Set of prompt rules that should be tried"""
    rules: List[Rule] = Field(
        ..., description="A list of prompt rules that we can try to solve for the instruction.")


class Promptnado:
    def __init__(self, system_prompt: str, instruction: str, examples: List[str], rule_token="<HERE>"):
        self.system_prompt = system_prompt
        self.instruction = instruction
        self.examples = examples
        self.rule_token = rule_token

        # Create random dataset name
        self.dataset_name = f"Promptnado_{random.randint(0, 1000000)}"

        self.attempts = 1
        self.solved = False

    def _create_dataset(self):
        """Create a dataset with a unique name"""
        dataset = client.create_dataset(
            self.dataset_name, description=self.instruction)
        for example in self.examples:
            client.create_example(
                inputs={"input": example}, dataset_id=dataset.id)

        self.dataset = dataset
        return dataset

    def _generate_rules(self):
        """Use an LLM to generate a list of rules"""

        system_prompt = f"""You are an expert LLM Prompt Engineer. Your job is to try to solve for the provided <Instructions> \
by making adjustments to the <Original Prompt>. You should attempt to make 5 suggestions for prompts that might work. Each suggestion you \
make will be interpolated into the prompt where {self.rule_token} is, and then evaluated for correctness against a dataset of \
examples.

<Instructions>
{self.instruction}
</Instructions>

<Original Prompt>
{self.system_prompt}
</Original Prompt>
"""

        structured_llm = init_chat_model(
            model="gpt-4o", temperature=0.7).with_structured_output(Rules)

        rules: Rules = structured_llm.invoke(system_prompt)

        self.rules = rules.rules
        return self.rules

    def _build_prompt(self, rule: Rule):
        """Interpolate the rules into the system prompt"""
        interpolated_prompt = self.system_prompt.replace(
            self.rule_token, rule.prompt)

        return interpolated_prompt

    def _evaluate_correctness(self, run: Run, example: Example):
        """Eval function to use an LLM to validate that the instruction was followed"""
        return {"score": 0, "key": "correctness"}

    def _predict(self, inputs: dict):
        """Predict the next rule"""

        messages = [
            SystemMessage(content=self.current_prompt),
            HumanMessage(content=inputs["input"]),
        ]

        # Invoke the model
        llm = init_chat_model(model="gpt-4o", temperature=0.7)
        response = llm.invoke(messages)

        return {"output": response.content}

    def _test_rule(self, rule: Rule):
        """Evaluate a given rule"""
        self.current_rule = rule

        self.current_prompt = self._build_prompt(self.current_rule)

        results = evaluate(
            self._predict,
            data=self.dataset_name,
            evaluators=[self._evaluate_correctness],
            experiment_prefix=f"Attempt-{self.attempts}",
        )

        self.attempts += 1

        return results

    def run(self):
        """Run the promptnado"""
        # Create the dataset
        self._create_dataset()

        while self.solved == False:
            # Get a list of rules
            self._generate_rules()

            # For each rule
            for rule in self.rules:
                results = self._test_rule(rule)

                # If the results are satisfactory, break
                if results["correctness"] > 0.9:
                    self.solved = True
                    break

        print("\n\nSolved!! Current prompt can be found at `self.current_prompt\n\n")

        print(f"Current prompt:\n\n{self.current_prompt}")